<a href="https://colab.research.google.com/github/Tamiyo22/AppointmentSchedulingApplication/blob/master/recent_earthquakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install folium pandas requests


In [12]:
import requests
import pandas as pd

url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson'
response = requests.get(url)
data = response.json()

# Normalize the JSON data to a pandas DataFrame
earthquakes = pd.json_normalize(data['features'])


In [13]:
# Extract relevant information
earthquakes['place'] = earthquakes['properties.place']
earthquakes['magnitude'] = earthquakes['properties.mag']
earthquakes['time'] = pd.to_datetime(earthquakes['properties.time'], unit='ms')
earthquakes['longitude'] = earthquakes['geometry.coordinates'].apply(lambda x: x[0])
earthquakes['latitude'] = earthquakes['geometry.coordinates'].apply(lambda x: x[1])


In [14]:
import folium

# Initialize a map centered around an average location
map_center = [earthquakes['latitude'].mean(), earthquakes['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=2)

# Add earthquake locations to the map
for idx, row in earthquakes.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['magnitude'],
        popup=f"{row['place']}, Mag: {row['magnitude']}",
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

# Save the map to an HTML file
m.save('earthquakes_map.html')

